In [1]:
import sys
import pandas as pd
import json

In [2]:
train_path = '../dataset/PIZZA_train.json'
test_path = '../dataset/PIZZA_dev.json'
def read_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            try:
                record = json.loads(line.strip())
                data.append(record)
                
                # Process in chunks of 10,000 records
                if i > 0 and i % 10000 == 0:
                    print(f"Processed {i} records so far...")
            except json.JSONDecodeError:
                continue
    return data

# Convert remaining data to DataFrame
data = read_data(train_path)
if data:
    df = pd.DataFrame(data)
    
data = read_data(test_path)
if data:
    dev = pd.DataFrame(data)

Processed 10000 records so far...
Processed 20000 records so far...
Processed 30000 records so far...
Processed 40000 records so far...
Processed 50000 records so far...
Processed 60000 records so far...
Processed 70000 records so far...
Processed 80000 records so far...
Processed 90000 records so far...
Processed 100000 records so far...
Processed 110000 records so far...
Processed 120000 records so far...
Processed 130000 records so far...
Processed 140000 records so far...
Processed 150000 records so far...
Processed 160000 records so far...
Processed 170000 records so far...
Processed 180000 records so far...
Processed 190000 records so far...
Processed 200000 records so far...
Processed 210000 records so far...
Processed 220000 records so far...
Processed 230000 records so far...
Processed 240000 records so far...
Processed 250000 records so far...
Processed 260000 records so far...
Processed 270000 records so far...
Processed 280000 records so far...
Processed 290000 records so f

In [3]:
from transformers import AutoTokenizer

# Example: Using a pre-trained tokenizer like BERT
src_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tgt_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

src_text = "can i have a large bbq pulled pork"
tgt_text = "(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING bbq pulled pork ) ) )"

# Tokenize the input and output
src_tokens = src_tokenizer(src_text, return_tensors="pt", padding=True, truncation=True)
tgt_tokens = tgt_tokenizer(tgt_text, return_tensors="pt", padding=True, truncation=True)

print(src_tokens)
print(tgt_tokens)

C:\Users\Hima\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'input_ids': tensor([[  101,  2064,  1045,  2031,  1037,  2312, 22861,  4160,  2766, 15960,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[  101,  1006,  2344,  2064,  1045,  2031,  1006, 10733,  8551,  2121,
          1006,  2193,  1037,  1007,  1006,  2946,  2312,  1007,  1006, 22286,
         22861,  4160,  2766, 15960,  1007,  1007,  1007,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])}


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader

class PizzaDataset(Dataset):
    def __init__(self, data, src_tokenizer, tgt_tokenizer):
        self.data = data
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_text = self.data["train.SRC"][idx]
        tgt_text = self.data["train.TOP"][idx]

        src_tokens = self.src_tokenizer(src_text, return_tensors="pt", padding="max_length", max_length=50, truncation=True)
        tgt_tokens = self.tgt_tokenizer(tgt_text, return_tensors="pt", padding="max_length", max_length=50, truncation=True)

        return {
            "src_input_ids": src_tokens["input_ids"].squeeze(0),
            "src_attention_mask": src_tokens["attention_mask"].squeeze(0),
            "tgt_input_ids": tgt_tokens["input_ids"].squeeze(0),
            "tgt_attention_mask": tgt_tokens["attention_mask"].squeeze(0),
        }


train_dataset = PizzaDataset(df, src_tokenizer, tgt_tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [5]:
train_dataset.__getitem__(1)

{'src_input_ids': tensor([  101,  2312, 11345,  2007,  2665, 11565,  1998,  2007,  4469, 27233,
         26534,  3490,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'src_attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]),
 'tgt_input_ids': tensor([  101,  1006,  2344,  1006, 10733,  8551,  2121,  1006,  2946,  2312,
          1007, 11345,  2007,  1006, 22286,  2665, 11565,  1007,  1998,  2007,
          1006,  3375,  1035, 22286,  1006, 11712,  4469,  1007,  1006, 22286,
         27233, 26534,  3490,  1007,  1007,  1007,  1007,   102,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [6]:
class DevPizzaDataset(Dataset):
    def __init__(self, data, src_tokenizer, tgt_tokenizer):
        self.data = data
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_text = self.data["dev.SRC"][idx]
        tgt_text = self.data["dev.TOP"][idx]

        src_tokens = self.src_tokenizer(src_text, return_tensors="pt", padding="max_length", max_length=50, truncation=True)
        tgt_tokens = self.tgt_tokenizer(tgt_text, return_tensors="pt", padding="max_length", max_length=50, truncation=True)

        return {
            "src_input_ids": src_tokens["input_ids"].squeeze(0),
            "src_attention_mask": src_tokens["attention_mask"].squeeze(0),
            "tgt_input_ids": tgt_tokens["input_ids"].squeeze(0),
            "tgt_attention_mask": tgt_tokens["attention_mask"].squeeze(0),
        }
val_dataset = DevPizzaDataset(dev, src_tokenizer, tgt_tokenizer)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [8]:
val_dataset.__getitem__(3)

{'src_input_ids': tensor([  101,  1045,  1005,  1040,  2066,  2000,  2344,  1037,  2312, 20949,
          1998, 11565, 10733,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'src_attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]),
 'tgt_input_ids': tensor([  101,  1006,  2344,  1045,  1005,  1040,  2066,  2000,  2344,  1006,
         10733,  8551,  2121,  1006,  2193,  1037,  1007,  1006,  2946,  2312,
          1007,  1006, 22286, 20949,  1007,  1998,  1006, 22286, 11565,  1007,
         10733,  1007,  1007,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [12]:
len(val_dataset.__getitem__(3)['src_input_ids'])

50

### BiLSTM-Based Transformer

In [9]:
import torch
import torch.nn as nn
from tqdm import tqdm

class BiLSTMEncoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, vocab_size, num_layers=1):
        super(BiLSTMEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = True
        self.num_directions = 2 if self.bidirectional else 1
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers,
                            bidirectional=self.bidirectional, batch_first=True)

    def forward(self, src, src_lengths):
        embedded = self.embedding(src)  # (batch_size, seq_len, embedding_dim)

        # Initialize hidden and cell states
        batch_size = src.size(0)
        h0 = torch.zeros(self.num_layers * self.num_directions, batch_size, self.hidden_dim).to(src.device)
        c0 = torch.zeros(self.num_layers * self.num_directions, batch_size, self.hidden_dim).to(src.device)

        # Pack the sequences for the LSTM
        src_lengths = src_lengths.cpu()  # Move lengths to CPU
        packed = nn.utils.rnn.pack_padded_sequence(embedded, src_lengths, batch_first=True, enforce_sorted=False)

        # Pass through the LSTM
        outputs, (hidden, cell) = self.lstm(packed, (h0, c0))
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)

        # Return all hidden states (both forward and backward)
        return outputs, hidden, cell



class LSTMDecoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, vocab_size, num_layers=1):
        super(LSTMDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(1)  # (batch_size, 1)
        embedded = self.embedding(input)  # (batch_size, 1, embedding_dim)
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(1))  # (batch_size, vocab_size)
        return prediction, hidden, cell


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, src_lengths, tgt, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        tgt_len = tgt.shape[1]
        tgt_vocab_size = self.decoder.fc_out.out_features

        # Tensor to store decoder outputs
        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        # Encode source sequence
        encoder_outputs, hidden, cell = self.encoder(src, src_lengths)

        # Reshape hidden and cell states for the decoder
        hidden = hidden.view(self.encoder.num_layers, batch_size, -1)
        cell = cell.view(self.encoder.num_layers, batch_size, -1)

        # First input to the decoder is the <sos> token
        input = tgt[:, 0]

        for t in range(1, tgt_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t, :] = output
            # Decide whether to use teacher forcing
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)  # Get the highest predicted token
            input = tgt[:, t] if teacher_force else top1

        return outputs

def train_model(model, dataloader, optimizer, criterion, device):
    model.train()
    epoch_loss = 0
    total_batches = len(dataloader)
    progress_bar = tqdm(dataloader, desc="Training Progress", unit="batch", leave=True)
    for batch_idx, batch in enumerate(progress_bar):
        src = batch["src_input_ids"].to(device)
        src_lengths = (batch["src_attention_mask"] != 0).sum(dim=1).cpu()
        tgt = batch["tgt_input_ids"].to(device)

        optimizer.zero_grad()
        output = model(src, src_lengths, tgt)
        # Reshape for loss computation
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        tgt = tgt[:, 1:].reshape(-1)

        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        avg_loss = epoch_loss / (batch_idx + 1)
        progress_bar.set_description(f"Training Progress: Batch {batch_idx + 1}/{total_batches}, Avg Loss: {avg_loss:.4f}")


    return epoch_loss / len(dataloader)

def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            src = batch["src_input_ids"].to(device)
            src_lengths = (batch["src_attention_mask"] != 0).sum(dim=1).to(device)
            tgt = batch["tgt_input_ids"].to(device)

            output = model(src, src_lengths, tgt, teacher_forcing_ratio=0)  # No teacher forcing during evaluation
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            tgt = tgt[:, 1:].reshape(-1)

            loss = criterion(output, tgt)
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)


### Initialize and Train

In [10]:
INPUT_DIM = len(src_tokenizer.vocab)
OUTPUT_DIM = len(tgt_tokenizer.vocab)
EMBEDDING_DIM = 256
HIDDEN_DIM = 512
NUM_LAYERS = 2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = BiLSTMEncoder(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, INPUT_DIM, NUM_LAYERS).to(DEVICE)
decoder = LSTMDecoder(EMBEDDING_DIM, HIDDEN_DIM * 2, OUTPUT_DIM, OUTPUT_DIM, NUM_LAYERS).to(DEVICE)
model = Seq2Seq(encoder, decoder, DEVICE).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=tgt_tokenizer.pad_token_id)


train_loss = train_model(model, train_dataloader, optimizer, criterion, DEVICE)
val_loss = evaluate_model(model, val_dataloader, criterion, DEVICE)
print(f"Train Loss: {train_loss:.3f}, Val Loss: {val_loss:.3f}")


Training Progress: Batch 1768/38382, Avg Loss: 1.3373:   5%|▍         | 1768/38382 [27:46<9:35:16,  1.06batch/s] 


KeyboardInterrupt: 

### Inference

In [ ]:
def generate(model, src, src_lengths, max_len=50):
    model.eval()
    with torch.no_grad():
        encoder_outputs, hidden, cell = model.encoder(src, src_lengths)
        input = torch.tensor([tgt_tokenizer.bos_token_id]).to(model.device)

        outputs = []
        for _ in range(max_len):
            output, hidden, cell = model.decoder(input, hidden, cell)
            top1 = output.argmax(1).item()
            outputs.append(top1)
            if top1 == tgt_tokenizer.eos_token_id:
                break
            input = torch.tensor([top1]).to(model.device)

        return tgt_tokenizer.decode(outputs, skip_special_tokens=True)


In [ ]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    print(f"Model loaded from {path}")
    return model

In [ ]:
save_model(model, "../weights/transformer_Bilstm.pt")